In [4]:
%pylab inline
from scipy import stats
from angles import r2d, r2arcs, d2arcs, arcs2r
def arcm2r(theta):
    return arcs2r(theta*60)
import seaborn as sns;sns.set_style('darkgrid')
import lsst.sims.maf.stackers as stackers
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19, 14
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
import lsst.sims.maf.maps as maps
from collections import defaultdict
from itertools import chain
sns.set_context('poster', font_scale=1.4)
import healpy as hp
import pickle
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

Populating the interactive namespace from numpy and matplotlib


In [8]:
def noebv(year=10, runName='baseline2018a'):
    proposalId = 3
    nights = year*365 + 1
    directory = '/global/cscratch1/sd/husni/OpsimRuns/'
    opsdb = db.OpsimDatabase(directory+runName+'.db')
    outDir = 'temp'
    resultsDb = db.ResultsDb(outDir=outDir)
    nside = 256
    myBundles = {}
    sqlWhere = 'night < '+str(nights)+' and \
    filter = "i" and proposalId = '+str(proposalId)

    slicer = slicers.HealpixSlicer(
        lonCol='fieldRA', latCol='fieldDec', nside=nside, useCache=False
        )

    metric = metrics.ExgalM5(lsstFilter='i')
    dustMap = maps.DustMap(interp=False, nside=nside)
    stackerList = [stackers.RandomDitherFieldPerVisitStacker(degrees=True)]

    myBundles['field'] = metricBundles.MetricBundle(metric, slicer, constraint=sqlWhere, 
                                                           stackerList=stackerList, runName=runName,
                                                           metadata='field dither', mapsList=[dustMap])
    bgroup = metricBundles.MetricBundleGroup(myBundles, opsdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll() 
    bundle = myBundles['field']
    cond = bundle.metricValues.mask==False
    area = len(bundle.metricValues[cond])*hp.pixelfunc.nside2pixarea(nside=nside, degrees=True)
    return area

In [9]:
def withebv(runName='baseline2018a', year=10):
    proposalId = 3
    nights = year*365 + 1
    directory = '/global/cscratch1/sd/husni/OpsimRuns/'
    opsdb = db.OpsimDatabase(directory+runName+'.db')
    outDir = 'temp'
    resultsDb = db.ResultsDb(outDir=outDir)
    nside = 256
    myBundles = {}
    sqlWhere = 'night < '+str(nights)+' and \
    filter = "i" and proposalId = '+str(proposalId)

    slicer = slicers.HealpixSlicer(
        lonCol='fieldRA', latCol='fieldDec', nside=nside, useCache=False
        )

    metric = metrics.ExgalM5(lsstFilter='i')
    dustMap = maps.DustMap(interp=False, nside=nside)
    stackerList = [stackers.RandomDitherFieldPerVisitStacker(degrees=True)]

    myBundles['field'] = metricBundles.MetricBundle(metric, slicer, constraint=sqlWhere, 
                                                           stackerList=stackerList, runName=runName,
                                                           metadata='field dither', mapsList=[dustMap])
    bgroup = metricBundles.MetricBundleGroup(myBundles, opsdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll() 
    bundle = myBundles['field']
    
    cond = np.logical_and(bundle.slicer.getSlicePoints()['ebv']<0.2, bundle.metricValues.mask==False)
    area = len(bundle.metricValues[cond])*hp.pixelfunc.nside2pixarea(nside=nside, degrees=True)
    return area

In [10]:
beforecut = noebv()

Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint night < 3651 and     filter = "i" and proposalId = 3 for columns ['fieldRA', 'fieldDec', 'fiveSigmaDepth']
Found 444867 visits
Running:  ['field']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [11]:
aftercut = withebv()

Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint night < 3651 and     filter = "i" and proposalId = 3 for columns ['fieldRA', 'fieldDec', 'fiveSigmaDepth']
Found 444867 visits
Running:  ['field']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [12]:
print(beforecut, aftercut, aftercut-beforecut)

18040.40708040273 14695.716992530577 -3344.6900878721517


In [13]:
def withebvdepth(runName='baseline2018a', year=10):
    proposalId = 3
    nights = year*365 + 1
    directory = '/global/cscratch1/sd/husni/OpsimRuns/'
    opsdb = db.OpsimDatabase(directory+runName+'.db')
    outDir = 'temp'
    resultsDb = db.ResultsDb(outDir=outDir)
    nside = 256
    myBundles = {}
    sqlWhere = 'night < '+str(nights)+' and \
    filter = "i" and proposalId = '+str(proposalId)

    slicer = slicers.HealpixSlicer(
        lonCol='fieldRA', latCol='fieldDec', nside=nside, useCache=False
        )

    metric = metrics.ExgalM5(lsstFilter='i')
    dustMap = maps.DustMap(interp=False, nside=nside)
    stackerList = [stackers.RandomDitherFieldPerVisitStacker(degrees=True)]

    myBundles['field'] = metricBundles.MetricBundle(metric, slicer, constraint=sqlWhere, 
                                                           stackerList=stackerList, runName=runName,
                                                           metadata='field dither', mapsList=[dustMap])
    bgroup = metricBundles.MetricBundleGroup(myBundles, opsdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll() 
    bundle = myBundles['field']
    vmin = 26
    cond = np.logical_and.reduce((bundle.slicer.getSlicePoints()['ebv']<0.2, bundle.metricValues.mask==False, bundle.metricValues.data>vmin))
    area = len(bundle.metricValues[cond])*hp.pixelfunc.nside2pixarea(nside=nside, degrees=True)
    return area

In [14]:
after2cuts = withebvdepth()

Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint night < 3651 and     filter = "i" and proposalId = 3 for columns ['fieldRA', 'fieldDec', 'fiveSigmaDepth']
Found 444867 visits
Running:  ['field']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [16]:
aftercut

14695.716992530577

In [15]:
print(after2cuts)

14691.205789187567
